In [ ]:
# Setup the Jupyter version of Dash
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import dash_table
from dash import callback_context
from dash.exceptions import PreventUpdate
from collections import Counter

import plotly.graph_objs as go
import plotly.express as px

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalCenterClass import AnimalShelter

# Set up the Jupyter version of Dash -- added this
#app = JupyterDash(__name__)

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# Handle empty strings and NaN values
unique_outcome_types = df['outcome_type'].unique()
unique_outcome_types = [str(outcome) for outcome in unique_outcome_types if pd.notna(outcome) and outcome != '']

# Print the updated unique outcome types
print("Updated unique outcome types:", unique_outcome_types)

# Generate options for the dropdown
options = [{'label': outcome, 'value': outcome} for outcome in unique_outcome_types]

# Print the options
print("Dropdown options:", options)

# Print the length of the options
print("Options length:", len(options))

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image

encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Center([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),title = 'Max Gilhespy'),
        html.B(html.H1('Max Gilhespy CS-340 Project Two')),
        html.Hr(),
        
        html.Label("Select Outcome Types"),
        dcc.Dropdown(
            id='outcome-type-dropdown',
            options=options,
            multi=True,
            value=[]  # Default value, i.e., no filter applied
        ),
        
        html.Button("Reset Filters", id="reset-filters-btn", n_clicks=0),
        
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            style_table={'height': '500px', 'overflowY': 'auto'},
            row_selectable='single',
            selected_rows=[0],
            sort_action='native',
            filter_action='native',
            page_action='native',
            page_size=10,
        ),
        html.Br(),
        html.Hr(),
        # Geolocation chart
        html.Div(
            id='map-id',
            className='col s12 m6',
            children=[
                dl.Map(style={'width': '100%', 'height': '500px'},
                       center=[df['location_lat'].mean(), df['location_long'].mean()], zoom=10, children=[
                           dl.TileLayer(id="base-layer-id"),
                           dl.Marker(position=[df['location_lat'][0], df['location_long'][0]],
                                     children=[
                                         dl.Tooltip(df['breed'][0]),
                                         dl.Popup([
                                             html.H1("Animal Name"),
                                             html.P(df['name'][0])
                                         ])
                                     ])
                       ])
            ]
        ),
        
        html.Br(),
        html.Div(className='row', style={'display': 'flex'}, children=[


            # Set up pie chart
            html.Div([
                dcc.Graph(
                    id='pie-chart-id',
                    figure=px.pie(df, names='outcome_type', title='Outcomes by Rescue Type')
                )
            ], className='col s12 m6')
            
            
        ])

    ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Check if selected_columns is None
    if selected_columns is None:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Define column names
    location_lat = "location_lat"
    location_long = "location_long"
    breed = "breed"
    name = "name"

    # Check if viewData is None or an empty list
    if viewData is None or not viewData:
        return []

    try:
        # Try to create DataFrame from viewData
        dff = pd.DataFrame.from_dict(viewData)
    except (TypeError, ValueError):
        print("Error creating DataFrame from viewData")
        return []

    # Check if a row is selected
    if index is None or len(index) == 0:
        # If no row is selected, set row to 0 or handle it based on your requirements
        row = 0
    else:
        # Use the first selected row
        row = index[0]

    # Check if the row index is within bounds
    if 0 <= row < len(dff):
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[dff[location_lat].mean(), dff[location_long].mean()], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(position=[dff.iloc[row][location_lat], dff.iloc[row][location_long]],
                                 children=[
                                     dl.Tooltip(dff.iloc[row][breed]),
                                     dl.Popup([
                                         html.H1("Animal Name"),
                                         html.P(dff.iloc[row][name])
                                     ])
                                 ])
                   ])
        ]
    else:
        # Handle the case where the index is out of bounds (customize based on your requirements)
        return []
    
@app.callback(
    Output('pie-chart-id', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    if not data:
        # If no data, return a default pie chart
        return px.pie(names=['Foster', 'Adopt', 'Euthanasia', 'Return to Owner', 'Transfer'],
                      values=[0, 0, 0, 0, 0], title='Outcomes by Rescue Type')

    df_selected = pd.DataFrame.from_dict(data)

    # Assuming 'outcome_type' is the column containing the outcome values
    outcome_counts = Counter(df_selected['outcome_type'])

    # Ensure that the outcome_counts is not empty
    if outcome_counts:
        total_outcomes = sum(outcome_counts.values())
        percentages = [(count / total_outcomes) * 100 for count in outcome_counts.values()]
        names = list(outcome_counts.keys())

        fig = px.pie(names=names, values=percentages, title='Percentage of Outcomes by Rescue Type')
        return fig

    return px.pie(names=['Foster', 'Adopt', 'Euthanasia', 'Return to Owner', 'Transfer'],
                  values=[0, 0, 0, 0, 0], title='Outcomes by Rescue Type')

@app.callback(
    Output('datatable-id', 'data'),
    [Input('outcome-type-dropdown', 'value'),
     Input('reset-filters-btn', 'n_clicks')],
    prevent_initial_call=True
)
def update_table(outcome_type_filters, reset_filters):
    print("Outcome type filters:", outcome_type_filters)
    ctx = callback_context

    # Handle the reset button
    triggered_component_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_component_id == 'reset-filters-btn':
        return df.to_dict('records')

    # Start with the original data
    filtered_df = df.copy()

    # Apply filters
    if outcome_type_filters:
        filtered_df = filtered_df[filtered_df['outcome_type'].isin(outcome_type_filters)]

    return filtered_df.to_dict('records')




# Run the Dash app
if __name__ =='__main__':
    app.run_server(mode="external", port=31889)